In [1]:
from threading import Thread
from time import sleep
import socket
import pickle
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Cipher import PKCS1_v1_5 as Cipher_PKCS1_v1_5
from Crypto.Hash import SHA256
from pprint import pprint

import os
import hashlib

HOST = "192.168.43.3"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)

In [2]:
def generate_RSA_key():
    keys = RSA.generate(2048)
    pub  = RSA.import_key(keys.public_key().export_key('DER'))
    prv  = RSA.import_key(keys.export_key('DER'))
    return prv,pub

def load_RSA_key():
    if(os.path.exists("userkey.pickle")):
        try:
            f=open("userkey.pickle","rb")
            prv,pub = pickle.load(f)
            prv = RSA.import_key(prv)
            pub = RSA.import_key(pub)
            return prv,pub
        except Exception as e:
            print(e)
            return None
    else:
        return None

def save_RSA_key(prv,pub):
    keyfile = open("userkey.pickle","wb")
    pickle.dump([prv.export_key(),pub.export_key()],keyfile)
    keyfile.close()
    print("Write success")
    return True

def encrypt(key, data):
    enc_data = Cipher_PKCS1_v1_5.new(key).encrypt(data)
    return enc_data

def decrypt(key,enc_data):
    '''
    Params
        key - Key for decryption of message
        enc - Encrypted message
    '''
    data = Cipher_PKCS1_v1_5.new(key).decrypt(enc_data,None)
    return data

def sign(keypair,data):
    digest = SHA256.new(data)
    signer = PKCS115_SigScheme(keypair)
    signature = signer.sign(digest)
    return signature


def verify(enc_message,signature,publickey):
    '''
    Params
        enc_message - Encrypted Message to be verified.
        signature - Signature of the message.
        publickey - Public of the Private key used to create signature
    '''
    data,enc_og,pubkey = enc_message,signature,publickey
    digest = SHA256.new(data)
    verifier = PKCS115_SigScheme(publickey)
    try:
        verifier.verify(digest, signature)
        return True
    except:
        print("Verify Fail")
        return True

def insertDB(dec_message,public_key,torurl):
    global db
    if(torurl not in db):
        db[torurl]={"messages":[],"pubkey":public_key}
    db[torurl]["messages"].append(dec_message)
    
def process_message(data):
    # Format [torurl,enc_message,signature,public_key]
    data = pickle.loads(data)
    print(data)
    torurl,enc_message,signature,public_key = data
    public_key = RSA.import_key(public_key)
    if(verify(enc_message,signature,public_key)):
        dec_message = decrypt(prv,enc_message)
        print(dec_message)
        insertDB(dec_message,public_key,torurl)
        print("Done!")
        return True
    else:
        print("Verification Failure")
        return False

# Python Server
def receiver_function():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("0.0.0.0", PORT))
        s.listen()
        while True:
            conn, addr = s.accept()
            with conn:
                # TODO: Threading might give better performance
                while True:
                    # Assuming 1024 bytes as max message size
                    data = conn.recv(1024)
                    if not data:
                        break
                    process_message(data)
    except KeyboardInterrupt:
        print("Bye..")
    
# Client code for the AnonChat server
def send_message(pub,torurl,message):
    remote_host = torurl # URL for reciver 
    PORT = 9999   # Port for AnonChat on reciever 
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((remote_host, PORT))
        enc_message = encrypt(pub,message.encode())
        signature = sign(prv,enc_message)
        s.send(pickle.dumps([HOST,enc_message,signature,pub.export_key()]))
        s.close()

In [3]:
db = {}
keys = load_RSA_key()
prv,pub = (None,None)
if(keys==None):
    print("Keys not avaiable, generating")
    prv,pub = generate_RSA_key()
    print("Saving keys")
    save_RSA_key(prv,pub)
else:
    prv,pub = keys
thread_rec = Thread(target=receiver_function)

In [4]:
thread_rec.start() # Start main reciver thread

In [5]:
def UI_Main():
    chosen_person = None
    while(True):
        try:
            inp = input(">").strip()
            if(inp == "$help"):
                print("Help Menu - Conrgatz you have discovered help menu")
                print("1) $menu for menu")
                print("2) $help for help")
                print("Type a message to chat")
            elif(inp == "$menu"):
                print("1) Contact list")
                print("2) Messages")
                print("3) New Contact")
                print("4) Export Public Key")
                print("5) Print DB")
                print("6) Exit")
                inp = input("Choice?[1]:").strip()
                if(inp == "1"):
                    pprint(db.keys())
                    inp = int(input(f"Choice?[{list(db.keys())[0]}]:").strip())
                    chosen_person = list(db.keys())[inp]
                elif(inp == "2"):
                    if(chosen_person):
                        pprint(db[chosen_person]["messages"])
                    else:
                        print("No selected contact, please select one.")
                elif(inp=="3"):
                    contact = input("Contact url:")
                    db[contact]={"messages":[],"pubkey":None}
                    inp = input("PublicKey:").strip('\' ').encode()
                    key = None
                    try:
                        key = RSA.import_key(inp)
                    except:
                        print("Import Failure! Please try again!")
                    db[contact]["pubkey"] = key
                elif(inp=="4"):
                    print(pub.export_key("OpenSSH").decode())
                elif(inp=="5"):
                    print(db)
                elif(inp=="6"):
                    break;
                else:
                    print("Try again!")
            else:
                if(chosen_person):
                    send_message(db[chosen_person]["pubkey"],chosen_person,inp)
                else:
                    print("No selected contact, please select one.")
        except KeyboardInterrupt:
            print("Exiting!")
            break
        except Exception as e:
            print(e)
            continue

In [ ]:
UI_Main()

>$menu
1) Contact list
2) Messages
3) New Contact
4) Export Public Key
5) Print DB
6) Exit
Choice?[1]:5
{}
>$menu
1) Contact list
2) Messages
3) New Contact
4) Export Public Key
5) Print DB
6) Exit
Choice?[1]:4
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDFZNe7E3ruW8//ESTvPDbRoSlZfFU8MDmnhKgA8pufxSyMTm54ApvxK0PHez2h3/cQl1w74neIGUBHko+XodQM00KJc+ZitNzmvY2uHGnHRD4a38huhnq6bG1w+2i6ZvY2Kh08+nWZUb00Psd9yVsyb5mY2AkAwYeIiM0cnzQ5s+KfEoLZpq78yQwvIrbJdL0p6Y/iN2HO9D6O2Sf/z9hTnMdgPAJvOF9kDc3qHCW0YHMx39SUNdjcMLENaM1FMkCvg2HZu348+ISexdvjr9PxZRtj+Do/ZNtWhdHHJTwzTQIoDYICmj56I/ou7GsBjdbjscqE/MvqbpJ+DRvN3Rnh
>$menu
1) Contact list
2) Messages
3) New Contact
4) Export Public Key
5) Print DB
6) Exit
Choice?[1]:3
Contact url:192.168.8.5
PublicKey:ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCbd5wIZBn5cnz1mn74WlKo8s0NkvDzLVnFXBaLqlBy4vBx7s6OVD7MGcsTKvg0E7pSoL5A5Eq2BlxgVOse0tlI+dJKQ/uuiyJ7zw3aP08wHs+94oxyMfeS07k+Pf0MLOSn2R1x2lLGdeR3zoLF6KkPDbfTE89ZUaYitSzDAloFB86GnDhbcMwfRcKUHQK0e1HQ1Pz3Xz54+hW1k3KLaEuwyaT8nV6ocAt8SdjmDS

In [14]:
keypair = generate_RSA_key()

In [16]:
mess = encrypt(keypair[1],"Hello".encode())
signed = sign(prv,mess)

In [17]:
signed

b'\xaf\x04\xe4\xa3\xfd\x9c$\r\xf8t\xa4\x9a\x84W\xeek\xc3N\x07\xefW$N+y\x81@\xf99/\x04\x95\xb8\x88\x96\xcd\xa7S\x90\xc7\xa73-\x80\xd8\xdbg\xe7\xd8A\x9c,r\x89\xb7\xb0j\xe5\xc1J\x04\x90@,\xf5\xb5\x93\xf4Gk0^\x13n\xa3\xff9x\xfb\x90\xb5\xb7\x11\x8f\xf2\x10\xd5\x13bs/\xa0\xb9\xebRT\x064\x9c\x19\xe1Y+\xc1;\x0e\x99\xca\x12\xb7P\x97\xd8\xcek\r\xc1\xb0^\xb3\x9d\x85k\xe2P\xdfU\no\x19\xaf\x1a@\x854\x7f\x86\xd5\xdbv\x08\x01W\xe7A\xf2\x17\xf7~\xa1P\x94\xb8\x87%\xa9~^\x88,\xf0\x99\xc6U.\x1fe\t\xf6\x93J\xaay\x1b\xcd\x94\xde8\xd0\x94\xe2\xbf\xd6*\xd9\x8df\xa6\x8f\xfbS\x80,_\x95%*\xbf\xe0\xe4pZ\xc7\xf2\xb6\x0e3\x97\x99wGM\xc5\x1e;\xd4U1\x1b\x1e(w\t\x0b\x05W\xf8\x8c(\xf1fcQ\x914V\x15\x8e\xa7/\x08\x8e\xcd4\xab\xb0#\xaf\x93T3QeI\xff\x19'

In [18]:
pub.export_key("OpenSSH")

b'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDFZNe7E3ruW8//ESTvPDbRoSlZfFU8MDmnhKgA8pufxSyMTm54ApvxK0PHez2h3/cQl1w74neIGUBHko+XodQM00KJc+ZitNzmvY2uHGnHRD4a38huhnq6bG1w+2i6ZvY2Kh08+nWZUb00Psd9yVsyb5mY2AkAwYeIiM0cnzQ5s+KfEoLZpq78yQwvIrbJdL0p6Y/iN2HO9D6O2Sf/z9hTnMdgPAJvOF9kDc3qHCW0YHMx39SUNdjcMLENaM1FMkCvg2HZu348+ISexdvjr9PxZRtj+Do/ZNtWhdHHJTwzTQIoDYICmj56I/ou7GsBjdbjscqE/MvqbpJ+DRvN3Rnh'

In [19]:
keypair[0].export_key("OpenSSH")

b'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCEGkPkgRSkw1wQF4vFns32RZhi0d74HFSvjtHITxOdbbImAbDJIh+2tCzYwHDvXElPiif4McWGbbDjWsZwXuDXs5gZj4KV4wAvD/CM58Jt1l/h2Phqvq7ZF0pe2aG5wJ0E0liqnrO/69K2ZnCl1HalBpzOKFO4ler20PicimJtwqkSKr7IEXj1gPGMQHGVjiPCgRmA/lwSQpThj37SJvq+H1dSAVAUBVt8FuIR7OMk01hDQ2NZmeEDdhYK+SwqWCrAr+7oTg3H1kRL+e61dNBdI7aEZ7eUSAKoTTPIouhmJ63np80TrnOOd4C4NjvgBMOUZYoL5yYZbgsvw3BTB1wT'

In [20]:
mess

b'gH*X\xbbT,\xd1r2i5\x88d\xaf\xb2\xe5\xed\xfc\tR\x1e\x83\xf5\xbf\xd5v"\xa5\x9bF\xc8\xf7l82G\x8aU\x84Q\xc8\x89\x0f\xc7\rR\x19\x05\n,Nxs\x13\xbe Z|\xb8\xcf\xdb/2 \xfe\xdft\xa3A\x94\xf8\xee\xad1)G\xab\x11\xda\x03UZ\xdah\xc8\xc7\xd3\xc8{\xc0B\xda\x8e\xdd\xcc\xd2B\xfe\x8a\xd9J\x96Z\xd6\xdd\xa5~\xe6ie\xf8\x86\x17\x81\x14}rc&\xba\xd0\xe7\x1d:\x17h\x92\x1ds\xcfx\xeaP\xd8\x03\xac)\xaf\x9a\xd17\xafb\xf9\xeeK\xb6\x05d\xa1\x98\xe9\x8d%O8\xacFe\x91r^\xcf\xb9 s\x91\x8f\xa6\xb0;\xc2\x1b\xfb|8o@\x9a[D&oa\x8b)\x91\x8b\x1a\xect\xe46\x86\xa0P\x8bQ_\x10\xe6U\xc5{Y\xedx)\xb2\x9d6\n\x82>4\xe8i\xe6\xbc8v\x16\xdaE\x06(\x86\x0cR\xc9\x14\xe1\xc9}F\x03x\x06`\t\xde\x88\xd8^\x8ay\x1f.\x1d\xaf3\\\xb2z-'